Implemented a Generator Class, which can generate vehicles and tasks for said vehicles with a poisson distribution.
The second to last cell can be manipulated to try out the FIFO/ EDF scheduling resource

In [1]:
! pip install simpy
import simpy
! pip install numpy
import numpy as np
import random
random.seed(42)

In [2]:
class Task:

    def __init__(self, id, priority, duration):
        self.id = id
        self.priority = priority
        self.duration = duration
        self.time_spent_processing = 0

    def process(self, environment, resource):

        print(f'Task {self.id} came into existence at {environment.now}')
        environment.plan.append(f'Task {self.id} came into existence at {environment.now} with priority {self.priority} and duration {self.duration}')
        
        # keep requesting resource if processing duration hasn't been reached
        while(self.time_spent_processing < self.duration):

             if isinstance(resource, simpy.PriorityResource):
                req = resource.request(priority = self.priority)
             else:
                req = resource.request()

             with req as req:
            
                try:
                    #before receiving resource
                    print(f'Task {self.id} requesting at {environment.now} with priority {self.priority}')
                    yield req
                    #while holding resource
                    print(f'Task {self.id} got resource at {environment.now}')
                    environment.plan.append(f'Task {self.id} got resource at {environment.now}')
                    #try to timeout for the time needed to complete the task
                    yield environment.timeout(self.duration - self.time_spent_processing) 
                    print(f'Task {self.id} completed at {environment.now}')
                    #exit loop if processing is complete
                    break

                except simpy.Interrupt as interrupt:
                    by = interrupt.cause.by
                    usage = environment.now - interrupt.cause.usage_since
                    #save the processing progress of the task
                    self.time_spent_processing += usage
                    print(f'Task {self.id} got preempted by {by} at {environment.now} after {usage} time entities')

In [3]:
class Vehicle:

    def __init__(self, id, environment,tasks,resource):

        self.id = id
        self.environment = environment
        self.tasks = tasks
        self.resource = resource

        print(f'Vehicle {self.id} spawned in environment at time {self.environment.now}')

        for task in tasks:                             #starts each tasks lifecycle
            self.environment.process(task.process(self.environment, resource))

    def allocateTask(self, task):
        self.tasks.append(task)
        self.environment.process(task.process(self.environment, self.resource))
        


Generate a vehicle around every 5 time units and a new task every 2.5 units

In [4]:
class Generator:

    def __init__(self, lambda_tasks, lambda_vehicles, env, resource):

        self.lambda_tasks = lambda_tasks
        self.lambda_vehicles = lambda_vehicles
        self.environment = env
        self.resource = resource
        self.vehicles = []
        self.tasks = []
        self.task_count = 0
        self.vehicle_count = 0
        self.event = simpy.Event(env)       #event for first car created 
        #self.plan = []

        env.process(self.generateVehicles())
        env.process(self.generateTasks())

    def generateTasks(self):
        
        while True:
         yield self.environment.timeout(np.random.poisson(self.lambda_tasks))
         priority = random.randint(0, 5)
         duration = random.randint(1, 5)
         new_task = Task(self.task_count, priority, duration)
         self.tasks.append(new_task)
         #print(f'created new task at time {self.environment.now}')     # first new task output was inconsistent
         self.task_count+=1
         # allocate to a vehicle
         if len(self.vehicles) == 0:
          yield self.event # make sure there is at least one vehicle
         random_vehicle = np.random.choice(self.vehicles)      
         random_vehicle.allocateTask(new_task)


    def generateVehicles(self):
        while True:
         yield self.environment.timeout(np.random.poisson(self.lambda_vehicles))
         new_vehicle = Vehicle(self.vehicle_count, self.environment,[], self.resource)
         self.vehicles.append(new_vehicle)

         if not self.event.triggered:
          self.event.succeed()

In [5]:
intersection = simpy.Environment()
intersection.plan = []

#different resources to try out for different scheduling types 
cpuEDF = simpy.PreemptiveResource(intersection, capacity = 1)
cpuFifo = simpy.Resource(intersection)

poissonVehicles = 5  #1 time interval per vehicle
poissonTasks = 2.5  #1 time interval per task

generator = Generator(poissonTasks, poissonVehicles, intersection, cpuEDF)     #switch out cpu resource for different scheduling results.

intersection.run(until = 30)



Vehicle 0 spawned in environment at time 5
Task 0 came into existence at 5
Task 0 requesting at 5 with priority 5
Task 0 got resource at 5
Task 0 completed at 6
Task 1 came into existence at 7
Task 1 requesting at 7 with priority 0
Task 1 got resource at 7
Task 2 came into existence at 9
Task 2 requesting at 9 with priority 1
Task 1 completed at 10
Task 2 got resource at 10
Task 3 came into existence at 11
Task 3 requesting at 11 with priority 1
Vehicle 0 spawned in environment at time 12
Task 2 completed at 12
Task 4 came into existence at 12
Task 4 requesting at 12 with priority 5
Task 3 got resource at 12
Task 5 came into existence at 13
Task 5 requesting at 13 with priority 0
Task 3 got preempted by <Process(process) object at 0x116ec3b50> at 13 after 1 time entities
Task 5 got resource at 13
Task 6 came into existence at 16
Task 6 requesting at 16 with priority 3
Vehicle 0 spawned in environment at time 17
Task 5 completed at 18
Task 6 got resource at 18
Task 7 came into existence

In [6]:
print(f"Number of Tasks: {generator.task_count}")
print(f"Number of Vehicles: {len(generator.vehicles)}")
print("Task scheduling plan:")
for step in intersection.plan:
    print(step)

Number of Tasks: 15
Number of Vehicles: 6
Task scheduling plan:
Task 0 came into existence at 5 with priority 5 and duration 1
Task 0 got resource at 5
Task 1 came into existence at 7 with priority 0 and duration 3
Task 1 got resource at 7
Task 2 came into existence at 9 with priority 1 and duration 2
Task 2 got resource at 10
Task 3 came into existence at 11 with priority 1 and duration 1
Task 4 came into existence at 12 with priority 5 and duration 5
Task 3 got resource at 12
Task 5 came into existence at 13 with priority 0 and duration 5
Task 5 got resource at 13
Task 6 came into existence at 16 with priority 3 and duration 1
Task 6 got resource at 18
Task 7 came into existence at 19 with priority 0 and duration 1
Task 7 got resource at 19
Task 4 got resource at 20
Task 8 came into existence at 21 with priority 1 and duration 2
Task 9 came into existence at 21 with priority 4 and duration 5
Task 8 got resource at 21
Task 10 came into existence at 22 with priority 0 and duration 5
Ta

To Do:

LIFO Resource,
plot scheduling plan,
give vehicle ids and print out task allocation to which car

